In [10]:
inputList = []
lockObj = []
transactionObj = []
waitingTransactions = []
returnflag = 1

In [11]:
#Class Name: lock
#We use this class to store locking details of transaction
class lock:
    def __init__(self, lockedDataItem, transactionID, lockState):
        self.lockedDataItem = lockedDataItem
        self.lockState = lockState
        self.waitingTransactions = []
        self.lockHeldBy = []
        self.lockHeldBy.append(transactionID)

    def addWaitingTransaction(self, transactionID):
        self.waitingTransactions.append(transactionID)

    def addLockHeld(self, transactionID):
        self.lockHeldBy.append(transactionID)

    def changeLockState(self, LS):
        self.lockState = LS

In [12]:
#Class Name: transaction
#We use this class to store transaction details like its state, timestamp.
class transaction():
    def __init__(self, transactionID, timestamp, transactionState):
        self.transactionID = transactionID
        self.timestamp = timestamp
        self.transactionState = transactionState
        self.lockedResoures = []
        self.blockedOperation = []
        # self.blockedOperation=blockedOperations
       
    def changeTransactionState(self, state):
        self.transactionState = state

    def addBlockedOperation(self, operation):
        self.blockedOperation.append(operation)

    def addLockedResource(self, resourceName):
        self.lockedResoures.append(resourceName)

In [13]:
#Function Name: searchTransactionID
#Input: transaction id
#Search transaction
def searchTransactionID(transactionID):
    for transaction in transactionObj:
        if transaction.transactionID == transactionID:
            return transaction

In [14]:
#Function Name: resourcenID
#Input: inputLine
#This function is called every time a new transaction is started
def resourcenID(inputLine):
    if inputLine.find("X") != -1:
        resourceName = "X"
    elif inputLine.find("Y") != -1:
        resourceName = "Y"
    elif inputLine.find("Z") != -1:
        resourceName = "Z"
    transactionID = getTransactionNo(inputLine)
    return resourceName, transactionID

In [15]:
#Input: inputLine
#Function Name: getTransactionNo
#This function is used to get the transaction number from the input string and return in digit
def getTransactionNo(str1):
    c = ""
    for i in str1:
        if i.isdigit():
            c += i
    return int(c)

In [16]:
#Function Name: begin
#Input: inputLine
#This function is called every time a new transaction is started
def begin(str):
    tranNumber = getTransactionNo(str)
    temp = int(len(transactionObj)) + 1
    transactionObj.append(transaction(tranNumber, temp, 'Active'))
    print ("{} Begin T{}: Record is added to transaction table with Tid={} and TS(T{})={}. T{} state=active".format(str,tranNumber,tranNumber,tranNumber,temp,tranNumber))

In [17]:
#Input: inputLine
#Function Name: read
#Assign resource for Read lock. Calls Wound - wait if deadlock occurs
def read(inputLine):
    resourceName = resourcenID(inputLine)[0]
    transactionID = resourcenID(inputLine)[1]
    #print(transactionID)
    flag = 0
    length = len(lockObj)
    if((transactionObj[transactionID-1].transactionState != 'Wait') and (transactionObj[transactionID-1].transactionState != 'Abort')):
        if length != 0:
            for i in range(0, length):
                if lockObj[i].lockedDataItem == resourceName:
                    flag = 1
                    # If the resource is under write Lock, Call  WaitDie
                    if lockObj[i].lockState == 'WriteLock':  
                        print("{} Because {} is write locked by transaction {}. Wait die mechanism is checked.".format(inputLine, resourceName, lockObj[i].lockedDataItem))
                        waitDie(searchTransactionID(transactionID),searchTransactionID(lockObj[i].lockHeldBy[0]), lockObj[i], inputLine)
                    elif lockObj[i].lockState == 'ReadLock':  
                        lockObj[i].addLockHeld(transactionID);
                        searchTransactionID(transactionID).addLockedResource(resourceName)
                        print ("{} {} Read locked by T{}: Lock table record for {} is created with mode R. (T{} holds the lock)".format(inputLine, resourceName,transactionID, resourceName,lockObj[i].lockHeldBy[0]))

            if flag == 0:
                lockObj.append(lock(resourceName, transactionID,'ReadLock')) 
                searchTransactionID(transactionID).addLockedResource(resourceName)
                print ("{} {} Read locked by T{}: Lock table record for {} is created with mode R. (T{} holds the lock)".format(inputLine, resourceName, transactionID, resourceName, transactionID))
        else:
            lockObj.append(lock(resourceName, transactionID, 'ReadLock'))
            searchTransactionID(transactionID).addLockedResource(resourceName)
            print ("{} {} Read locked by T{}: Lock table record for {} is created with mode R. (T{} holds the lock)".format(inputLine, resourceName,transactionID, resourceName,lockObj[0].lockHeldBy[0]))
    elif(transactionObj[transactionID-1].transactionState == 'Abort'):
        print("{} T{} = Already Aborted".format(inputLine,transactionID))
    else:
        print("{} Because T{} is blocked. We add this to list of waiting operation of T{} -> {}".format(inputLine,transactionID, transactionID, inputLine))

In [18]:
#Input: inputLine
#Function Name: write
#Assign resource for Write lock. Calls Wound - wait if deadlock occurs
def write(inputLine):
    resourceName = resourcenID(inputLine)[0]
    transactionID = resourcenID(inputLine)[1]
    flag = 0
    length = len(lockObj)
    if(transactionObj[transactionID-1].transactionState == 'Abort'):
        print("{} T{} = Already Aborted".format(inputLine,transactionID))
    else:
        if length != 0:
            for i in range(0, length):
                if lockObj[i].lockedDataItem == resourceName:
                    flag = 1
                    if lockObj[i].lockState == 'ReadLock':
                        if len(lockObj[i].lockHeldBy) == 1:
                            if lockObj[i].lockHeldBy[0] == transactionID:
                                # checks to see if the resources is under readLock
                                lockObj[i].lockState = 'WriteLock'
                                print("{} Read lock upgraded to write lock for item {} by T{}, lock table updated to mode W.".format(inputLine, resourceName,transactionID))
                            else:
                                # call Wait die if its under readlock by another transaction
                                print("{} Because {} is read locked by another transaction. Wait die mechanism is checked".format(inputLine, resourceName))
                                waitDie(searchTransactionID(transactionID),searchTransactionID(lockObj[i].lockHeldBy[0]), lockObj[i],
                                          inputLine)
                        else:
                            count = 0
                            for lockedresource in lockObj:
                                if lockedresource.lockedDataItem == resourceName:
                                    for tempheldby in lockedresource.lockHeldBy:
                                        if tempheldby == transactionID:
                                            count += 1

                            print("{} Because {} is read locked by transactions {}. Wait die mechanism is checked.".format(inputLine, resourceName,lockObj[i].lockHeldBy))
                            waitDie(searchTransactionID(transactionID),searchTransactionID(lockObj[i].lockHeldBy[count]), lockObj[i],inputLine)
                    elif lockObj[i].lockState == 'WriteLock':
                        print("{} Because {} is Write locked. Wait die mechanism is checked.".format(inputLine, resourceName))
                        waitDie(searchTransactionID(transactionID),searchTransactionID(lockObj[i].lockHeldBy[0]), lockObj[i], inputLine)
            if flag == 0:
                lockObj.append(lock(resourceName, transactionID, 'WriteLock'))
                searchTransactionID(transactionID).addLockedResource(resourceName)
        else:
            lockObj.append(lock(resourceName, transactionID, 'WriteLock'))
            searchTransactionID(transactionID).addLockedResource(resourceName)

In [19]:
#Input: requestingTransaction, holdingTransaction, lockedResource, operation
#Function Name: waitDie
#Perform wait die
def waitDie(requestingTransaction, holdingTransaction, lockedResource, operation):
    if (requestingTransaction.timestamp < holdingTransaction.timestamp):
        # Adds the transaction to the waitingTransactions list
        requestingTransaction.changeTransactionState('Wait')  
        print ("changing transaction state for T{} to blocked".format(requestingTransaction.transactionID))
        if checkDuplicateOperation(operation, requestingTransaction):
            requestingTransaction.addBlockedOperation(operation)
        if checkDuplicateTransaction(requestingTransaction):
            waitingTransactions.append(requestingTransaction)
    else:
        requestingTransaction.changeTransactionState('Abort')
        print ("Aborting T{}" .format(requestingTransaction.transactionID))
       # holdingTransaction.changeTransactionState('Wait')
        holdingTransaction.addBlockedOperation(operation)
        waitingTransactions.append(requestingTransaction)
        resourceUnlock(requestingTransaction.transactionID)  

In [20]:
#Input: transaction
#Function Name: checkDuplicateTransaction
#Check if the transaction is in the Waiting List
def checkDuplicateTransaction(transaction):
    for t in waitingTransactions:
        if t.transactionID == transaction.transactionID:
            return 0
    return 1

#Input: operation, transaction
#Function Name: checkDuplicateOperation
#Check if the transaction is in the Blocked transaction List
def checkDuplicateOperation(operation, transaction):
    for blockedOperation in transaction.blockedOperation:
        if blockedOperation == operation:
            return 0
    return 1

In [21]:
#Input: operation
#Function Name: checkOperationState
#This function checks the state of the transaction
def checkOperationState(operation):
    resourceName = resourcenID(operation)[0]
    transactionID = resourcenID(operation)[1]

    length = len(transactionObj)
    if length != 0:
        for i in range(0, length):
            if transactionObj[i].transactionID == transactionID and transactionObj[i].transactionState == 'Wait':
                transactionObj[i].addBlockedOperation(operation)
            elif transactionObj[i].transactionState == 'Abort':
                operation = ""
                print(transactionID + " is Aborted")
    return operation

In [22]:
#Input: transactionID
#Function Name: resourceUnlock
#This function unlocks the resuorces of the transaction Id
def resourceUnlock(transactionID):
    #print ("Resources held by T{} are unlocked".format(transactionID))
    for transaction in transactionObj:
        if transaction.transactionID == transactionID:
            for lock in transaction.lockedResoures:
                for resource in lockObj:
                    if resource.lockedDataItem == lock:
                        if len(resource.lockHeldBy) == 1:
                            # Remove this transaction from the list of transactions holding the lock
                            lockObj.remove(resource)  
                            print("Data item {} is unlocked".format(resource.lockedDataItem))
                        else:
                            # Remove the resource from the lockTable completely.
                            try:
                                resource.lockHeldBy.remove(transactionID) 
                                print("Data item {} is unlocked".format(resource.lockedDataItem))
                            except ValueError:
                                pass # or scream: thing not in some_list!
                             
    WaitingTransactions()

In [23]:
#Function Name: WaitingTransactions
#Input: transactionID
#This function checks if any transactions waiting can be resumed
import copy
def WaitingTransactions():
    for transaction in waitingTransactions:
        if transaction.transactionState == 'Abort':
            waitingTransactions.remove(transaction)
        else:
            blockOpCopy = copy.deepcopy(transaction.blockedOperation)
            for blockedOperation in transaction.blockedOperation:
                if(transaction.transactionState != 'Active'):
                    print("Activate T{}. Set T{} = active".format(transaction.transactionID, transaction.transactionID))
                    transaction.transactionState = 'Active'  # we activate the transaction in the waitingTransactions list and pull the operations that are in the waitlist
                    print ("We now execute the waiting operation " + blockedOperation)
                    print("\n")
                    FunctionAssignment(blockedOperation)  # call the FunctionAssignment method on the waiting operation and see if the transaction can now continue
                    if transaction.transactionState != 'Wait':
                        blockOpCopy.remove(blockedOperation)
            transaction.blockedOperation = blockOpCopy
            if len(transaction.blockedOperation) == 0:
                waitingTransactions.remove(transaction)

In [24]:
#Function Name: commit
#Input: operation
#This function is called when the transaction ends
def commit(op):
    tranNumber = getTransactionNo(op)
    for transaction in transactionObj:
        try:
            if transaction.transactionID == tranNumber and transaction.transactionState != 'Abort':
                print("{} T{} = Commit. Releases all locks held by T{}".format(op, tranNumber, tranNumber))
                transaction.transactionState = 'Commit'
            elif transaction.transactionID == tranNumber and transaction.transactionState == 'Abort':
                print("{} T{} = Already aborted".format(op, tranNumber))
        except AttributeError:
           #print("{} T{} = {}. Releases all locks held by T{}".format(op, tranNumber,transaction.transactionState, tranNumber))
            transaction.transactionState = 'Commit'
    resourceUnlock(tranNumber)

In [25]:
#Input: operation read from input file
#Function Name: FunctionAssignment
#Perform respective operations
def FunctionAssignment(op):
    if op.find('b') == 1:
        op= checkOperationState(op)
    if op.find('b') != -1:
        begin(op)
    elif op.find('r') != -1:
        read(op)
    elif op.find('w') != -1:
        write(op)
    elif op.find("e") != -1:
        commit(op)
#Reading file from user input as per assignment
file=input("Enter the input filename (For eg: input1.txt):")
if file.endswith('.txt'):
    with open(file, 'r') as text:
        for line in text:
            inputList.append(line)
        for inputStr in inputList:
            print("\n")
            FunctionAssignment(inputStr)
else:
    print('Error: Please use .txt extension')

Enter the input filename (For eg: input1.txt):input1.txt


b1;
 Begin T1: Record is added to transaction table with Tid=1 and TS(T1)=1. T1 state=active


r1 (Y);
 Y Read locked by T1: Lock table record for Y is created with mode R. (T1 holds the lock)


r1 (Z);
 Z Read locked by T1: Lock table record for Z is created with mode R. (T1 holds the lock)


b2;
 Begin T2: Record is added to transaction table with Tid=2 and TS(T2)=2. T2 state=active


r2 (Y);
 Y Read locked by T2: Lock table record for Y is created with mode R. (T1 holds the lock)


b3;
 Begin T3: Record is added to transaction table with Tid=3 and TS(T3)=3. T3 state=active


r3 (Y);
 Y Read locked by T3: Lock table record for Y is created with mode R. (T1 holds the lock)


w1 (Z);
 Read lock upgraded to write lock for item Z by T1, lock table updated to mode W.


w3 (Y);
 Because Y is read locked by transactions [1, 2, 3]. Wait die mechanism is checked.
Aborting T3
Data item Y is unlocked


w2 (Y); 
 Because Y is read locked